In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

import string
import nltk
import pymorphy2

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy.lib.function_base import vectorize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

import pickle

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# 1. Загрузка данных

In [4]:
with open('train.json', encoding = 'utf-8') as json_file:
    data = json.load(json_file)

Будем решать задачу классификации на 2 класса

In [5]:
print(set([x["label"] for x in data]))

{'обеспечение гарантийных обязательств', 'обеспечение исполнения контракта'}


# 2. Предобработка и токенизация

In [6]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# Посмотрим на стоп-слова
stop_words[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [7]:
# знаки препинания
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Инициализируем WordPunctTokenizer, с помощью которого затем разобьём текст на слова

In [8]:
word_tokenizer = nltk.WordPunctTokenizer()

In [9]:
dates = [str(x) for x in np.arange(1900, 2022)]

def process_data(data):
    """
    Предобработка текста
    """
    texts = []
    targets = []
    
    # поочередно проходим по всем словам в списке
    for item in data:
               
        text_lower = item['text'].lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) # разбиваем текст на слова
        
        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and word not in dates)]
        
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [10]:
# запускаем нашу предобработку
%%time
y = [item['label'] for item in data]
texts = process_data(data)

CPU times: user 1.96 s, sys: 22.6 ms, total: 1.98 s
Wall time: 2.04 s


Теперь каждый пример представлен списком слов. Причем все слова с прописной буквы. Пунктуацию и стоп-слова мы удалили

In [11]:
i = 159
print("Label: ", y[i])
print("Tokens: ", texts[i][:5])

Label:  обеспечение исполнения контракта
Tokens:  ['приложение', '№', '4', 'извещению', 'осуществлении']


# 3. Нормализация слов

In [12]:
morph = pymorphy2.MorphAnalyzer()

Взглянем что получается

In [13]:
i = 1
for aword in texts[i][:10]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s.\tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: требования.	Лемматизированное: требование
Исходное слово: содержанию.	Лемматизированное: содержание
Исходное слово: заявки.	Лемматизированное: заявка
Исходное слово: участника.	Лемматизированное: участник
Исходное слово: запроса.	Лемматизированное: запрос
Исходное слово: котировок.	Лемматизированное: котировка
Исходное слово: электронной.	Лемматизированное: электронный
Исходное слово: форме.	Лемматизированное: форма
Исходное слово: заявка.	Лемматизированное: заявка
Исходное слово: участие.	Лемматизированное: участие


Отлично, теперь применим лемматизацию ко всему документу

In [14]:
for i in tqdm_notebook(range(len(texts))):
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]]
    texts[i] = ' '.join(text_lemmatized) 

  0%|          | 0/1799 [00:00<?, ?it/s]

# 4. Моделирование и векторные представления

Лейблы у нас закодированы словами. Для корректной работы алгоритма конвертируем их в числа

('обеспечение гарантийных обязательств', 'обеспечение исполнения контракта'):

```
обеспечение гарантийных обязательств = 0
обеспечение исполнения контракта = 1

```

In [15]:
def label2num(y):
    """
    Функция для кодирования лейблов
    """
    if y == 'обеспечение гарантийных обязательств':
        return 0
    if y ==  'обеспечение исполнения контракта':
        return 1

encoded_y = [label2num(label) for label in y]

Разобъём на train и test

In [16]:
train_texts, test_texts, train_y, test_y = train_test_split(texts, encoded_y, test_size=0.2, random_state=42, stratify=y)

# TF-IDF

In [17]:
vectorizer = TfidfVectorizer(max_features=200, norm = None) # возмем топ 200 слов
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names_out()[:20]

array(['00', '04', '05', '10', '11', '12', '13', '14', '16', '2022', '30',
       '44', '45', '95', '96', '___', '____', 'аванс', 'адрес', 'акт'],
      dtype=object)

In [18]:
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

Посмотрим что получается в первых двух строчках

In [19]:
train_X.todense()[:2]

matrix([[ 2.10278766,  0.        ,  0.        ,  1.94446161,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  1.84429312,
          0.        ,  3.4199086 ,  2.00709718,  0.        ,  0.        ,
          2.25380518,  2.70405389,  0.        ,  0.        ,  0.        ,
          3.82564905,  1.87546874,  2.113303  ,  2.07391968,  0.        ,
          0.        ,  3.21853139,  2.52300541,  0.        ,  0.        ,
          0.        ,  5.19916857,  0.        ,  0.        ,  0.        ,
          3.04128978,  0.        ,  3.82911225,  0.        ,  2.94445665,
          0.        ,  0.        , 31.66502907,  0.        ,  2.1006978 ,
          3.53024136,  0.        ,  0.        ,  7.04413652,  3.50945424,
          0.        ,  3.0684918 ,  2.04388739,  2.24653359,  0.        ,
          0.        ,  0.        ,  0.        ,  2.89250104,  0.        ,
          0.        ,  0.        , 15.73268788,  0.        ,  5.23281316,
          1.86882419,  0.        ,  0.

# Обучаем классификатор

In [20]:
clf = RandomForestClassifier(n_estimators=500, max_features='sqrt', max_depth=10)
clf = clf.fit(train_X, train_y)

pred = clf.predict(test_X)

In [21]:
print('Предсказанные метки: ', pred[0:20], ".....")
print('Истинные метки: ', test_y[0:20], ".....")

Предсказанные метки:  [1 0 1 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1] .....
Истинные метки:  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1] .....


In [22]:
def num2label(y):
    """
    Функция для декодирования лейблов
    """
    if y == 0:
        return 'обеспечение гарантийных обязательств'
    if y == 1:
        return 'обеспечение исполнения контракта'

In [23]:
decoded_pred = [num2label(y) for y in pred]
decoded_test_y = [num2label(y) for y in test_y]

print('Предсказанные метки: ', decoded_pred[0:4])
print('Истинные метки: ', decoded_test_y [0:4])

Предсказанные метки:  ['обеспечение исполнения контракта', 'обеспечение гарантийных обязательств', 'обеспечение исполнения контракта', 'обеспечение гарантийных обязательств']
Истинные метки:  ['обеспечение исполнения контракта', 'обеспечение гарантийных обязательств', 'обеспечение исполнения контракта', 'обеспечение гарантийных обязательств']


Качество классификатора будем оценивать по метрике **accuracy**

In [24]:
print(accuracy_score(test_y, pred))

0.9638888888888889


Из любопытства взглянем на матрицу ошибок (**confusion_matrix**)

In [25]:
print(confusion_matrix(test_y, pred))

[[156   6]
 [  7 191]]


6 ошибок первого рода и 7 ошибок второго рода

Посмотрим на **roc_auc_score**

In [26]:
print(roc_auc_score(test_y, pred))

0.9638047138047138


Сохраним модель

In [27]:
with open("rf_clf.pkl", "wb") as f:
    pickle.dump(clf, f)

# Посмотрим на несколько примеров

In [28]:
for i in range(10):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст: ', train_texts[i][:100]+'...')
    print('\n')

Истинный лейбл: обеспечение исполнения контракта
Предсказанный лейбл: обеспечение исполнения контракта
Текст:  приложение № 2 документация аукцион электронный форма проект договор договор № поставка товар « ____...


Истинный лейбл: обеспечение гарантийных обязательств
Предсказанный лейбл: обеспечение гарантийных обязательств
Текст:  утверждать генеральный директор акционерный общество « объединить энергетический компания » быть про...


Истинный лейбл: обеспечение исполнения контракта
Предсказанный лейбл: обеспечение исполнения контракта
Текст:  32 11 приложение № 3 аукционный документация договор поставка № г москва « ___ » ___________ 20__г с...


Истинный лейбл: обеспечение гарантийных обязательств
Предсказанный лейбл: обеспечение гарантийных обязательств
Текст:  государственный контракт № _________ выполнение работа осуществление строительный контроль выполнени...


Истинный лейбл: обеспечение гарантийных обязательств
Предсказанный лейбл: обеспечение гарантийных обязательств
Текст

# LogisticRegression

In [29]:
lr_clf = LogisticRegression(n_jobs=-1, random_state=42).fit(train_X, train_y)

pred = lr_clf.predict(test_X)

In [30]:
def metrics(test_y, pred):
  print(f'accuracy_score:\n{accuracy_score(test_y, pred)}\n')
  print(f'confusion_matrix:\n{confusion_matrix(test_y, pred)} \n')
  print(f'roc_auc_score:\n{roc_auc_score(test_y, pred)}')

In [31]:
metrics(test_y, pred)

accuracy_score:
0.9222222222222223

confusion_matrix:
[[150  12]
 [ 16 182]] 

roc_auc_score:
0.9225589225589225


# LGBMClassifier

In [32]:
parameters = {'n_estimators': [5, 10, 50, 100, 500],
              'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
              'subsample' : [0.5, 0.7, 1.0],
              'max_depth': [3, 5, 7, 9]}

lgbm_clf = LGBMClassifier()
lgbm = GridSearchCV(lgbm_clf, 
                  parameters,
                  cv=3,
                  n_jobs=-1,
                  verbose=True,
                  scoring='accuracy')

lgbm.fit(train_X, train_y)
pred = lgbm.predict(test_X)

print('Best parameters: ', lgbm.best_params_)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
Best parameters:  {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.5}


In [33]:
%%time
lgbm = LGBMClassifier(learning_rate=0.01,
                     max_depth=3,
                     n_estimators=500,
                     subsample=0.5)

lgbm = lgbm.fit(train_X, train_y)

pred = lgbm.predict(test_X)

CPU times: user 606 ms, sys: 8.82 ms, total: 615 ms
Wall time: 320 ms


In [34]:
metrics(test_y, pred)

accuracy_score:
0.95

confusion_matrix:
[[155   7]
 [ 11 187]] 

roc_auc_score:
0.9506172839506173


In [36]:
test_df = pd.read_json('test.json')
test_df.head(3)

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
